-READ the README to understand the data
It tells you the column order
readme.txt isn’t a table — it’s just plain text. so i cant use pandas read

In [4]:
with open('../data/CMAPSSData/readme.txt','r',encoding='latin1') as file:
    cont=file.read()
print(cont)

Data Set: FD001
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: ONE (HPC Degradation)

Data Set: FD002
Train trjectories: 260
Test trajectories: 259
Conditions: SIX 
Fault Modes: ONE (HPC Degradation)

Data Set: FD003
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: TWO (HPC Degradation, Fan Degradation)

Data Set: FD004
Train trjectories: 248
Test trajectories: 249
Conditions: SIX 
Fault Modes: TWO (HPC Degradation, Fan Degradation)



Experimental Scenario

Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine  i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. Th

| File    | Used for                            |
| ------- | ----------------------------------- |
| `train` | Learn from full run-to-failure data |
| `test`  | Predict how long engines will last  |
| `RUL`   | Ground truth for evaluation         |

here i am not using 'RUL' because i am performing unsupervised learning

Encoding Types:
| Encoding                     | What It Does                                           | Good For                              |
| ---------------------------- | ------------------------------------------------------ | ------------------------------------- |
| **UTF-8**                    | Modern, universal, supports emoji, all languages       | Internet, Python default           |
| **Latin-1** (aka ISO-8859-1) | Old-school, supports Western European languages        | Older Windows files                |
| **cp1252**                   | Microsoft's fancy version of Latin-1 with curly quotes | MS Word files, PDFs, weird hyphens |


What Happened :
You opened a .txt file with encoding='utf-8'(default)

But the file was likely saved using Latin-1 or cp1252

So Python sees some bytes and goes “WTH is 0x96?!” 

In [5]:
import pandas as pd

file='../data/CMAPSSData/train_FD001.txt'

column_names = (
    ["engine_id", "cycle", "op_setting_1", "op_setting_2", "op_setting_3"] +
    [f"sensor_{i}" for i in range(1, 27)]  
)

df=pd.read_csv(file,sep='\s+',header=None,names=column_names)

df.head()

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\HP\AppData\Local\Temp\ipykernel_8960\337535694.py:10: SyntaxWarning: invalid escape sequence '\s'
  df=pd.read_csv(file,sep='\s+',header=None,names=column_names)


,engine_id,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,392,2388,100.0,39.06,23.4190,NaN,NaN,NaN,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,392,2388,100.0,39.00,23.4236,NaN,NaN,NaN,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,390,2388,100.0,38.95,23.3442,NaN,NaN,NaN,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,392,2388,100.0,38.88,23.3739,NaN,NaN,NaN,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,393,2388,100.0,38.90,23.4044,NaN,NaN,NaN,NaN,NaN


lefts perform on FD001 first,

-Only one fault mode

-Only one environment

-Clean, no noise from different ops

-Perfect for: Clustering, DBSCAN, basic RUL